# 第三章 LangChain 表达式语言 LangChain Expression Language

在这一章我们会介绍 LangChain Expression Language（或称为 LCEL），被称之为 Langchain 的表达式语言。LCEL 是一种新的语法，是 LangChain 工具包的重要补充，他有许多优点，使得我们处理 LangChain 和代理更加简单方便。

1. LCEL 提供了异步、批处理和流处理支持，这使得代码多功能化，并且代码可以快速在不同服务器中应用和运行。
    - 异步：程序可以同时执行多个任务，而不是按照顺序一个接一个地执行
    - 批处理：是一种将一组任务或数据作为一个批次进行处理的方法，而不是逐个处理
    - 流式处理：理是一种连续处理数据的方法，数据会持续不断地进入系统并被处理，流式处理能够在数据到达时立即进行处理，并且可以以持续且低延迟的方式处理数据。

2. LCEL 拥有 fallbacks 措施，也叫回退安全机制，有时LLM得到的结果不可控，这时你可以将结果进行回退，甚至可以附加到整个链上

3. LCEL 增加了 LLM 的并行性，LLM 运行通常是耗费时间的，并行可以加快得到结果的速度。

4. LCEL 内置了日志记录，记录代理的运行情况。即使代理复杂，日志也有助于理解复杂链条和代理的运行情况。

在前面的课程中，我们知道 LangChain 提供了组件 链（chain） 可以将组件组合起来发挥 LLM 更强大的功能，但是语法非常复杂。在这里，LCEL 提供了一种管道语法，使从基本组件构建复杂链变得容易，我们可以通过 LangChain 完成`Chain = prompt | LLM |OutputParser `的组合，具体使用我们将在下文内容中讨论。链（Chains）通常将大语言模型（LLM）与提示（Prompt）结合在一起，基于此，我们可以对文本或数据进行一系列操作。

![image.png](../../figures/LCEL.png)

- [一、简单链 Simple Chain](#一、简单链-Simple-Chain)
- [二、更复杂的链 More complex chain](#二、更复杂的链-More-complex-chain)
  - [1.1 构建简单向量数据库](#1.1-构建简单向量数据库)
  - [1.2 使用RunnableMap](#1.2-使用RunnableMap)
- [三、绑定 Bind](#三、绑定-Bind)
  - [3.1 单函数绑定](#3.1-单函数绑定)
  - [3.2 多个函数绑定](#3.2-多个函数绑定)
- [四、后备措施 Fallbacks](#四、后备措施-Fallbacks)
  - [4.1 使用早期模型格式化输出](#4.1-使用早期模型格式化输出)
  - [4.2 使用新模型格式化输出](#4.2-使用新模型格式化输出)
  - [4.3 fallbacks方法](#4.3-fallbacks方法)
- [五、接口 Interface](#五、接口-Interface)
  - [5.1 invoke接口](#5.1-invoke接口)
  - [5.2 batch接口](#5.2-batch接口)
  - [5.3 stream接口](#5.3-stream接口)
  - [5.4 异步接口](#5.4-异步接口)
- [六、英文版提示](#六、英文版提示)

为了大家实验方便，关于以下示例，我们将使用硅基流动的API，于是相关的代码也会修改。

## 一、简单链 Simple Chain

接下来我们依旧会使用 OpenAI 的 API，所以首先我们要初始化我们的 API_Key，这个方法和上一章的方式是一样的。

In [ ]:
# !pip install langchain
# !pip install openai==0.28
# !pip install "langchain[docarray]"
# !pip install tiktoken

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

接下来首先导入 LangChain 的库，并且定义一个简单的链，这个链包括提示模板，大语言模型和一个输出解析器。我们可以看到，成功输出了大语言模型的结果，完成了一个简单的链。

In [3]:
# 导入LangChain所需的模块
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# 使用 ChatPromptTemplate 从模板创建一个提示，模板中的 {topic} 将在后续代码中替换为实际的话题
prompt = ChatPromptTemplate.from_template(
    "告诉我一个关于{topic}的短笑话"
)

# 创建一个 ChatOpenAI 模型实例，默认使用 Qwen/Qwen3-8B 模型
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                   openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                   seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                   extra_body={
                       "enable_thinking": False
                   }
                  )

# 创建一个StrOutputParser实例，用于解析输出
output_parser = StrOutputParser()

# 创建一个链式调用，将 prompt、model 和output_parser 连接在一起
chain = prompt | model | output_parser

# 调用链式调用，并传入参数
print(chain.invoke({"topic": "熊"}))

当然可以！这里有一个关于熊的短笑话：

有一天，一只熊走进了一家酒吧，点了一杯啤酒，然后说：“我只想安静地喝一杯。”

酒保看了看它，说：“抱歉，我们这里不欢迎熊。”

熊耸耸肩，说：“那我换个地方，我只想安静地喝一杯。”

酒保无奈地说：“你真是个爱喝的熊。” 😄

希望你喜欢这个笑话！如果想要更多，我也可以继续讲。


如果我们去查看`Chain`的输出，我们会发现，他跟我们定义的是一样的，一共有三部分进行组成，也就是`Chain = prompt | LLM |OutputParser `。`|`符号类似于 unix 管道操作符，它将不同的组件链接在一起，将一个组件的输出作为输入提供给下一个组件。在这个链中，用户输入被传递给提示模板，然后提示模板输出被传递给模型，然后模型输出被传递到输出解析器。

In [4]:
# 查看Chain的值
chain

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='告诉我一个关于{topic}的短笑话'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AE22E1F4D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AE233237A0>, root_client=<openai.OpenAI object at 0x000001AE2121AF90>, root_async_client=<openai.AsyncOpenAI object at 0x000001AE23280CE0>, model_name='Qwen/Qwen3-8B', temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.siliconflow.cn/v1', max_retries=3, presence_penalty=0.1, frequency_penalty=0.1, seed=42, max_tokens=4096, extra_body={'enable_thinking': False})
| StrOutputParser()

## 二、更复杂的链 More complex chain

接下来，我们会创建一个更复杂的链条，在之前的课程中，我们接触过如何进行检索增强生成。所以接下来我们使用 LCEL 来重复之前的过程，将用户的问题和向量数据库检索结果结合起来，使用 RunnableMap 来构建一个更复杂的链。

### 2.1 构建简单向量数据库
首先我们构建一个向量数据库，这个简单的向量数据库只包含两句话，使用 OpenAI 的 Embedding 作为嵌入模型，然后我们通过 `vector store.as_retriever `来创建一个检索器。

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

embedding_model = OpenAIEmbeddings(
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    model="BAAI/bge-m3"
)

# 创建一个DocArrayInMemorySearch对象，用于存储和搜索文档向量
vectorstore = DocArrayInMemorySearch.from_texts(
    ["哈里森在肯肖工作", "熊喜欢吃蜂蜜"],
    embedding=embedding_model
)

# 创建一个检索器
retriever = vectorstore.as_retriever()

E:\LearningDisk\Learning_Projects\MyPythonProjects\llm-cookbook\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


通过之前的学习，如果我们调用`retriever.get_relevant_documents`，我们会得到相关的检索文档，首先我们问“哈里森在哪里工作？”，我们会发现返回了一个文档列表，他会根据相似度排序返回文档列表，所以其中最相关的放在了第一个。

In [7]:
# 获取与问题“哈里森在哪里工作？”相关的文档
retriever.invoke("哈里森在哪里工作？")

[Document(metadata={}, page_content='哈里森在肯肖工作'),
 Document(metadata={}, page_content='熊喜欢吃蜂蜜')]

如果我们换一个问题，比如"熊喜欢吃什么"，可以看到问题的顺序就发生了变化。

In [8]:
# 获取与问题“熊喜欢吃什么”相关的文档
retriever.invoke("熊喜欢吃什么")

[Document(metadata={}, page_content='熊喜欢吃蜂蜜'),
 Document(metadata={}, page_content='哈里森在肯肖工作')]

### 3.2 使用RunnableMap

上述例子返回两个结果是因为只有两个文档列表，这完全适用于更多文档情况。接下来我们会加入`RunnableMap`，在这个`RunnableMap`中，不仅仅有用户的问题，以及有对应的问题的文档列表，相当于这也为大模型的文档增加了上下文，这样就能完成检索增强的事情。如果我们正常问一个问题，可以看到，大模型正确的返回了文档里面的结果，得到了正确的输出。

In [9]:
from langchain.schema.runnable import RunnableMap

# 定义一个模板字符串template
template = """仅根据以下上下文回答问题：
{context}

问题：{question}
"""

# 使用 template 作为模板
prompt = ChatPromptTemplate.from_template(template)

# 创建一个处理链 chain ，包含了 RunnableMap、prompt、model 和 output_parser 组件
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

# 调用chain的invoke方法
chain.invoke({"question": "哈里森在哪里工作?"})

'哈里森在肯肖工作。'

如果我们想更深入挖掘一下背后的工作机理，我们可以看一下`RunnableMap`，我们把其创建为一个输入，用一样的方式进行操作。我们可以看到，在这之中，`RunnableMap`提供了`context`和`question`两个变量，一个是查询的文档列表，另一个是对应的问题，这个大模型就可以根据提供文档来总结回答对应的问题了。

In [10]:
# 创建一个RunnableMap对象，其中包含两个键值对
# 键 "context" 对应一个lambda函数，用于获取相关文档，函数输入参数为x，即输入的字典，函数返回值为retriever.get_relevant_documents(x["question"])
# 键 "question" 对应一个lambda函数，用于获取问题，函数输入参数为x，即输入的字典，函数返回值为x["question"]
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

# 调用 inputs 的 invoke 方法，并传递一个字典作为参数，字典中包含一个键值对，键为"question"，值为"哈里森在哪里工作?"
inputs.invoke({"question": "哈里森在哪里工作?"})

{'context': [Document(metadata={}, page_content='哈里森在肯肖工作'),
  Document(metadata={}, page_content='熊喜欢吃蜂蜜')],
 'question': '哈里森在哪里工作?'}

## 三、绑定 Bind

在上一章我们介绍了OpenAI函数的调用，新的`function`参数可以自动判断是否要使用工具函数，如果需要就会返回需要使用的参数。接下来我们也使用LangChain实现OpenAI函数调用的新功能，首先需要一个函数的描述信息，以及定义函数，这里的函数还是使用上一章的`get_current_weather`函数。

In [16]:
# 定义一个函数
get_current_weather_fn = {
    "name": "get_current_weather",
    "description": "获取指定位置的当前天气情况",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "城市和省份，例如：北京，北京市",
        },
        "unit": {"type": "string", "enum": ["摄氏度", "华氏度"]},
      },
      "required": ["location"],
    },
}

### 3.1 单函数绑定

接下来我们使用`bind`的方法把工具函数绑定到大模型上，并构建一个简单的链。进行调用以后，我们可以看到返回了一个`AIMessage`，	其中返回的`content`为空，但是返回了我们需要调用工具函数的参数。

In [17]:
# 使用ChatPromptTemplate.from_messages方法创建一个ChatPromptTemplate对象
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

# 使用bind方法绑定functions参数
model = model.bind(tools=[
    {
        "type": "function",
        "function": get_current_weather_fn
    }
])

runnable = prompt | model

# 调用invoke方法
runnable.invoke({"input": "北京天气怎么样？"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f1a44717c4182a160a787b51d6c', 'function': {'arguments': ' {"location": "北京", "unit": "摄氏度"}', 'name': 'get_current_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 197, 'total_tokens': 212, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f1a438bc4e812dd7a110a207b82', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4a897e02-fdb5-4e0f-9b30-42577e3d0b6e-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': '北京', 'unit': '摄氏度'}, 'id': '01975f1a44717c4182a160a787b51d6c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 15, 'total_tokens': 212, 'input_token_details': {}, 'output_token_details': {}})

### 3.2 多个函数绑定

同时我们也可以定义多个`function`，大模型在对话的时候可以自动判断使用哪一个函数。这里面我们定义有两个函数，第一个函数是类似于前面的`weather_search`，搜索给定机场的天气，然后我们还定义了一个赛事体育新闻搜索的`sports_search`，查询天气的函数`weather_search`接受的参数为airport_code即机场代码，体育新闻搜索函数`sports_search`接受的参数为team_name即体育队名。由于这里我们不需要运行这些函数，因为大模型是通过问的问题来自动判断是否调用这些函数，并且返回参数，并不会直接帮我们调用。

In [18]:
weather_search_fn = {
    "name": "weather_search",
    "description": "搜索给定机场代码的天气",
    "parameters": {
        "type": "object",
        "properties": {
            "airport_code": {
                "type": "string",
                "description": "要获取天气的机场代码"
            },
        },
        "required": ["airport_code"]
    }
}

sports_search_fn = {
    "name": "sports_search",
        "description": "搜索最近体育赛事的新闻",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "要搜索的体育队名"
                },
            },
            "required": ["team_name"]
        }
}

接着我们就可以使用函数绑定大模型，定义一个简单的链，我们可以看到，当我们问了相关的问题以后，大模型能够自动判断并且正确返回参数，知道需要调用函数了。

In [19]:
# 绑定大模型
model = model.bind(tools=[
    {
        "type": "function",
        "function": weather_search_fn
    },
    {
        "type": "function",
        "function": sports_search_fn
    }
])

runnable = prompt | model

runnable.invoke({"input": "爱国者队昨天表现的怎么样?"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f1b61c1160a4502c5183ac1a75d', 'function': {'arguments': ' {"team_name": "爱国者队"}', 'name': 'sports_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 251, 'total_tokens': 261, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f1b60c0d9eb77dbdb940a2abd98', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e82fe1d3-4a66-42bd-ab02-15b900982007-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': '爱国者队'}, 'id': '01975f1b61c1160a4502c5183ac1a75d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 251, 'output_tokens': 10, 'total_tokens': 261, 'input_token_details': {}, 'output_token_details': {}})

## 四、后备措施 Fallbacks

在使用早期的OpenAI模型如"text-davinci-001"，这些模型在对话过程中，不支持格式化输出结果即它们都是以字符串的形式输出结果，这对我们有时候需要解析 LLM 的输出带来一些麻烦，比如下面这个例子，就是利用早期模型"text-davinci-001"来回答用户的问题，我们希望 llm 能以 json 格式输出结果。

我们定义了 OpenAI 的模型以及创建了一个简单的链，以此加入 json 希望能以 json 格式输出结果，我们让 simple_model 写三首诗，并以 josn 格式输出，每首诗必须包含:`标题，作者和诗的第一句`。我们会发现结果只有字符串，无法输出指定格式的内容，虽然里面有一些`[`，但是本质上还是一个大的字符串，这就无法让我们解析输出。

> 由于OpenAI于2024年1月4日停用了模型text-davinci-001，你将使用OpenAI推荐的替代模型gpt-3.5-turbo-instruct。

在使用语言模型时，你可能经常会遇到来自底层 API 的问题，无论这些问题是速率限制还是停机时间。因此，当你将 LLM 应用程序转移到实际生产环境中时，防范这些问题变得越来越重要。这就是为什么我们引入了`回退（Fallbacks）`的概念。

### 4.1 使用早期模型格式化输出

In [16]:
from langchain.llms import OpenAI
import json

# 使用早期的OpenAI模型
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

challenge = "写三首诗，并以josn格式输出，每首诗必须包含:标题，作者和诗的第一句。"

simple_model.invoke(challenge)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


'\n\n{\n  "title": "春风",\n  "author": "李白",\n  "first_line": "春风又绿江南岸",\n  "content": [\n    "春风又绿江南岸",\n    "花开满树柳如丝",\n    "鸟儿欢唱天地宽",\n    "人间春色最宜人"\n  ]\n}\n\n{\n  "title": "夜雨",\n  "author": "杜甫",\n  "first_line": "夜雨潇潇",\n  "content": [\n    "夜雨潇潇",\n    "孤灯照旧",\n    "思念如潮",\n    "泛滥心头"\n  ]\n}\n\n{\n  "title": "山行",\n  "author": "王维",\n  "first_line": "远上寒山石径斜",\n  "content": [\n    "远上寒山石径斜",\n    "白云生处有人家",\n    "停车坐爱枫林晚",\n    "霜叶红于二月花"\n  ]\n}'

如果我们使用`simple_chain`来运行，我们就会发现出现了 json 解码错误的问题，因为返回的结果就是一个字符串，无法解析，所以下面代码就会报错。

In [17]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 15 column 1 (char 147)

### 4.2 使用新模型格式化输出

所以我们会发现早期版本的 OpenAI 模型不支持格式化的输出，所以即使使用 LangChain 并且加上了`json.load`但是还是会出现错误，但是如果我们使用新的`gpt-3.5-turbo`模型就不会出现这个问题。

In [27]:
import json

# 默认使用新的模型
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                   openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                   seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                   extra_body={
                       "enable_thinking": False
                   }
                  )

chain = model | StrOutputParser() | json.loads

challenge = "写三首诗，并以josn格式输出，不要输出其他内容，每首诗必须包含:标题，作者和诗的第一句。"

chain.invoke(challenge)

{'poems': [{'title': '秋思', 'author': '李商隐', 'first_line': '枯藤老树昏鸦，小桥流水人家'},
  {'title': '静夜思', 'author': '李白', 'first_line': '床前明月光，疑是地上霜'},
  {'title': '登鹳雀楼', 'author': '王之涣', 'first_line': '白日依山尽，黄河入海流'}]}

### 4.3 fallbacks方法

那这个时候可能就会思考，有没有什么方法，在不用改变太多代码的情况下，让早期的模型也能达到格式化输出的效果，而不是写复杂的格式化输出的代码去对结果进行操作。这时候我们就可以使用`fallbacks`的方式赋予早期模型这样格式化的能力，从结果我们也可以看出，我们成功使用`fallbacks`赋予了简单模型格式化的能力。

In [19]:
# 使用with_fallbacks机制
final_chain = simple_chain.with_fallbacks([chain])

# 调用final_chain的invoke方法，并传递challenge参数
final_chain.invoke(challenge)

{'poem1': {'title': '春风',
  'author': '李白',
  'first_line': '春风又绿江南岸。',
  'content': '春风又绿江南岸，明月何时照我还。'},
 'poem2': {'title': '静夜思',
  'author': '杜甫',
  'first_line': '床前明月光，',
  'content': '床前明月光，疑是地上霜。'},
 'poem3': {'title': '登鹳雀楼',
  'author': '王之涣',
  'first_line': '白日依山尽，',
  'content': '白日依山尽，黄河入海流。'}}

### 4.4 fallbacks 是如何实现的？

当我们调用 LLM 时，经常会出现由于底层 API 问题、速率问题或者网络问题等原因，导致不能成功运行 LLM 。在这种情况下，我们就可以使用回退这种方法来解决这个问题，具体来说，他是通过使用另一种 LLM 来代替原先的不可运行的 LLM 产生结果，请看下面例子：

In [ ]:
from langchain_core.chat_models.openai import ChatOpenAI
from langchain_core.chat_models.anthropic import ChatAnthropic

model = ChatAnthropic().with_fallbacks([ChatOpenAI()])
model.invoke('hello')

在这种情况下，通常会优先使用 ChatAnthropic 进行回答，但是如果调用 ChatAnthropic 失败了，会回退到使用 ChatOpenAI 模型来生成响应。如果两种 LLM 都失败了，将会回退到一种硬编码响应。硬编码的默认响应用于处理异常情况或者在无法从外部资源获取所需信息时提供一个备用选项，例如 "Looks like our LLM providers are down. Here's a nice 🦜️ emoji for you instead."（看起来我们的 LLM 提供商出了问题，那么，这里有一个可爱的 🦜️ 表情符号给你。）

如果你想了解更多关于 fallbacks 的内容，请参考[官方文档](https://python.langchain.com/docs/guides/fallbacks)

## 五、接口 Interface

在使用LangChain中，存在许多接口，其中公开的标准接口包括：

- stream：流式返回输出内容
- invoke：输入调用chain
- batch：在输入列表中并行调用chain

这些也有相应的异步方法：

- astream：异步流式返回输出内容
- ainvoke：在输入上异步调用chain
- abatch：在输入列表中并行异步调用chain

首先我们定义给一个简单提示模板，也就是"给我讲一个关于{主题}的短笑话"，然后定义了一个简单的链`Chain = prompt | LLM | OutputParser`。

In [28]:
# 创建一个ChatPromptTemplate对象，使用模板"给我讲一个关于{topic}的短笑话"
prompt = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的短笑话"
)

# 创建一个ChatOpenAI模型
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                   openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                   seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                   extra_body={
                       "enable_thinking": False
                   }
                  )

# 创建一个StrOutputParser对象
output_parser = StrOutputParser()

# 创建一个chain，将prompt、model和output_parser连接起来
chain = prompt | model | output_parser

### 5.1 invoke接口

接下来我们分别使用对应的接口，比如我们首先使用常规的`invoke`的调用，这个也是前面展现的方法，我们得到了对应结果。

In [29]:
chain.invoke({"topic": "熊"})

'当然可以！这里有一个关于熊的短笑话：\n\n有一天，一只熊走进了一家酒吧，点了一杯啤酒，然后说：“我只想安静地喝一杯。”\n\n酒保看了看它，说：“抱歉，我们这里不欢迎熊。”\n\n熊耸耸肩，说：“那我换个地方，我只想安静地喝一杯。”\n\n酒保无奈地说：“你总是这样，熊。”\n\n熊回答：“那我换个身份，我只想安静地喝一杯。”\n\n酒保：“……你到底是谁？”\n\n熊：“我是‘我只想安静地喝一杯’。”\n\n酒保：“……你真是个哲学熊。” 😄\n\n这个笑话有点文字游戏的意味，但希望你喜欢！如果你想要更搞笑或更温馨的版本，也可以告诉我。'

### 5.2 batch接口

我们再尝试使用`batch`的接口，我们会发现大模型可以返回两个问题的答案，我们会给chain一个输入的列表，列表中可以包含多个问题，最后返回多个问题的答案。

In [30]:
chain.batch([{"topic": "熊"}, {"topic": "狐狸"}])

['当然可以！这里有一个关于熊的短笑话：\n\n有一天，一只熊走进了一家酒吧，点了一杯啤酒，然后说：“我只想安静地喝一杯。”\n\n酒保看了看它，说：“抱歉，我们这里不欢迎熊。”\n\n熊耸耸肩，说：“那我换个地方，我只想安静地喝一杯。”\n\n酒保无奈地说：“你总是这样，熊。”\n\n熊回答：“那我换个身份，我只想安静地喝一杯。”\n\n酒保：“……你到底是谁？”\n\n熊：“我是‘我只想安静地喝一杯’。”\n\n酒保：“……你真是个哲学熊。” 😄\n\n这个笑话有点文字游戏的意味，但希望你喜欢！如果你想要更搞笑或更温馨的版本，也可以告诉我。',
 '当然可以！这里有一个关于狐狸的短笑话：\n\n有一天，一只狐狸走进了理发店，对理发师说：“我想剪个发型。”\n\n理发师问：“你想剪成什么样子？”\n\n狐狸回答：“我想剪成‘狐狸’的样子。”\n\n理发师惊讶地问：“那是什么样子的？”\n\n狐狸笑着说：“就是……像我这样！”']

### 5.3 stream接口

接下来我们在看看`stream`接口，也就是流式输出内容，这样的功能很有必要，有时候可以免去用户等待的烦恼，让用户看到一个一个词蹦出来而不是一个空的屏幕，这样会带来更好的用户体验。

In [31]:
for t in chain.stream({"topic": "熊"}):
    print(t)


当然
可以
！
这里
有一个
关于
熊
的
短
笑话
：


有一天
，
一只
熊
走进
了一
家
酒吧
，
点
了一
杯
啤酒
，
然后
说
：“
我
只想
安静
地
喝
一杯
。”


酒
保
看了看
它
，
说
：“
抱歉
，
我们
这里
不
欢迎
熊
。”


熊
耸
耸
肩
，
说
：“
那
我
换个
地方
，
我
只想
安静
地
喝
一杯
。”


酒
保
无奈
地说
：“
你
总是
这样
，
熊
。”


熊
回答
：“
那
我
换个
身份
，
我
只想
安静
地
喝
一杯
。”


酒
保
：“
……
你
到底
是谁
？
”


熊
：“
我是
‘
我
只想
安静
地
喝
一杯
’
。”


酒
保
：“
……
你
真是
个
哲学
熊
。”
 😄



这个
笑话
有点
文字
游戏
的
意味
，
但
希望
你喜欢
！
如果你
想要
更
搞笑
或
更
温馨
的
版本
，
也可以
告诉我
。



### 5.4 异步接口

我们还可以尝试异步来调用，使用`ainvoke`来调用。


In [32]:
response = await chain.ainvoke({"topic": "熊"})
response

'当然可以！这里有一个关于熊的短笑话：\n\n有一天，一只熊走进了一家酒吧，点了一杯啤酒，然后说：“我只想安静地喝一杯。”\n\n酒保看了看它，说：“抱歉，我们这里不欢迎熊。”\n\n熊耸耸肩，说：“那我换个地方，我只想安静地喝一杯。”\n\n酒保无奈地说：“你总是这样，熊。”\n\n熊回答：“那我换个身份，我只想安静地喝一杯。”\n\n酒保：“……你到底是谁？”\n\n熊：“我是‘我只想安静地喝一杯’。”\n\n酒保：“……你真是个哲学熊。” 😄\n\n这个笑话有点文字游戏的意味，但希望你喜欢！如果你想要更搞笑或更温馨的版本，也可以告诉我。'

## 六、英文提示词

**一、构建简单链**

In [33]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                   openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                   seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                   extra_body={
                       "enable_thinking": False
                   }
                  )
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})

"Why don't bears ever get cold?  \nBecause they always have *paws* in the snow! ❄️🐻"

**2.1 构建简单文档数据库**

In [35]:
embedding_model = OpenAIEmbeddings(
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    model="BAAI/bge-m3"
)

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()

In [36]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [37]:
retriever.invoke("what do bears like to eat")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [39]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

**3.2 使用RunnableMap**

In [40]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

**3.1 单函数绑定**

In [34]:
weather_search_fn {
  "name": "weather_search",
  "description": "Search for weather given an airport code",
  "parameters": {
    "type": "object",
    "properties": {
      "airport_code": {
        "type": "string",
        "description": "The airport code to get the weather for"
      },
    },
    "required": ["airport_code"]
  }
}

In [41]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = model.bind(tools=[
    {
        "type": "function",
        "function": weather_search_fn
    },
])

runnable = prompt | model

runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f257c4ff746c35bd2427ef2e53e', 'function': {'arguments': ' {"airport_code": "sf"}', 'name': 'weather_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 175, 'total_tokens': 183, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f257b2320284d4fbb03b6fc47ae', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6245c15e-d178-41cf-80cc-2180bb04e54e-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'sf'}, 'id': '01975f257c4ff746c35bd2427ef2e53e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 175, 'output_tokens': 8, 'total_tokens': 183, 'input_token_details': {}, 'output_token_details': {}})

**3.2 多个函数绑定**

In [42]:
weather_search = {
  "name": "weather_search",
  "description": "Search for weather given an airport code",
  "parameters": {
    "type": "object",
    "properties": {
      "airport_code": {
        "type": "string",
        "description": "The airport code to get the weather for"
      },
    },
    "required": ["airport_code"]
  }
}
sports_search_fn = {
  "name": "sports_search",
  "description": "Search for news of recent sport events",
  "parameters": {
    "type": "object",
    "properties": {
      "team_name": {
        "type": "string",
        "description": "The sports team to search for"
      },
    },
    "required": ["team_name"]
  }
}

In [43]:
model = model.bind(tools=[
    {
        "type": "function",
        "function": weather_search_fn
    },
    {
        "type": "function",
        "function": sports_search_fn
    }
])

runnable = prompt | model

runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '01975f268b31eb50420366c9eec21aab', 'function': {'arguments': ' {"team_name": "Patriots"}', 'name': 'sports_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 252, 'total_tokens': 262, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '01975f268a6fe8d58e92cedb8afa1587', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3b98e3a5-66b2-4c3e-8558-f60ee06b616a-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': 'Patriots'}, 'id': '01975f268b31eb50420366c9eec21aab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 252, 'output_tokens': 10, 'total_tokens': 262, 'input_token_details': {}, 'output_token_details': {}})

**4.1 使用早期模型格式化输出**

In [38]:
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

**早期模型不支持，会出现解码错误**

In [39]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

**4.2 较新的模型能够格式化输出**

In [45]:
model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                   openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                   seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                   extra_body={
                       "enable_thinking": False
                   }
                  )

chain = model | StrOutputParser() | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

chain.invoke(challenge)

{'poems': [{'title': 'The Road Not Taken',
   'author': 'Robert Frost',
   'first_line': 'Two roads diverged in a yellow wood,'},
  {'title': 'Still I Rise',
   'author': 'Maya Angelou',
   'first_line': 'You may write me down in history'},
  {'title': 'Ode to a Nightingale',
   'author': 'John Keats',
   'first_line': 'My heart aches, and aches, and still I rise.'}]}

**4.3 fallback机制**

In [41]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

**五、接口**

In [46]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

**5.1 invoke接口**

In [47]:
chain.invoke({"topic": "bears"})

"Why don't bears play hide and seek with their friends?  \nBecause good luck finding them—they’re always *bear* ing the fruit! 🐻🍎"

**5.2 batch接口**

In [48]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears play hide and seek with their cubs?  \nBecause the cubs always find them—*bear* is the best game ever! 🐻😄",
 "Why don't frogs ever get cold?  \nBecause they always carry a *toad* (towel) with them! 🐸🧺"]

**5.3 stream接口**

In [49]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 play
 hide
 and
 seek
 with
 their
 friends
?
  

Because
 good
 luck
 finding
 them
—they
’re
 always
 *
bear
*
 ing
 the
 fruit
!
 🐻
🍎



**5.4 异步接口**

In [50]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears play hide and seek with their friends?  \nBecause good luck finding them—they’re always *bear* ing the fruit! 🐻🍎"